# Imputación básica

## Preparar datos

In [187]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import random
import warnings
#import math
import auxi

## Leer datos originales

Hay que leer el dataset original para comparar los datos imputados con los reales

Temperatura

In [188]:
warnings.filterwarnings('ignore', category=SyntaxWarning)  # Porque el \d de \data y \daily lo toma como un valor especial
dataframe_temperatura = pd.read_csv("..\data\daily-minimum-temperatures-in-me.csv", header=0, index_col=0, parse_dates=True)
series_temperatura = dataframe_temperatura.squeeze('columns')
series_temperatura = series_temperatura.resample('D').mean()
series_temperatura['1984-12-31'] = (series_temperatura['1984-12-30'] + series_temperatura['1985-01-01'])/2
series_temperatura['1988-12-31'] = (series_temperatura['1988-12-30'] + series_temperatura['1989-01-01'])/2
series_temperatura.head()

Date
1981-01-01    20.7
1981-01-02    17.9
1981-01-03    18.8
1981-01-04    14.6
1981-01-05    15.8
Freq: D, Name: Daily minimum temperatures in Melbourne, Australia, 1981-1990, dtype: float64

Cardiovascular

In [189]:
dataframe_cardiovascular = pd.read_csv("..\data\\ads_hour.csv", header=0, index_col=0, parse_dates=True)
series_cardiovascular = dataframe_cardiovascular.squeeze('columns')
series_cardiovascular.head()

C:\Users\alaba\AppData\Local\Temp\ipykernel_23056\1350282516.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dataframe_cardiovascular = pd.read_csv("..\data\\ads_hour.csv", header=0, index_col=0, parse_dates=True)


Date
2017-08-03 00:00:00    49136
2017-08-03 01:00:00    46450
2017-08-03 02:00:00    46355
2017-08-03 03:00:00    43748
2017-08-03 04:00:00    42281
Name: ads, dtype: int64

Manchas solares

In [190]:
dataframe_sunspots = pd.read_csv("..\data\monthly-sunspots.csv", header=0, index_col=0, parse_dates=True)
series_sunspots = dataframe_sunspots.squeeze('columns')
series_sunspots = series_sunspots.resample('ME').mean() # Para los meses
series_sunspots.head()

Month
1749-01-31    58.0
1749-02-28    62.6
1749-03-31    70.0
1749-04-30    55.7
1749-05-31    85.0
Freq: ME, Name: Sunspots, dtype: float64

## Funciones auxiliares

### Media

In [191]:
def imputar_media(serie, ruta, lista):
    dataframe_media = pd.DataFrame({
        'Dataset': [],
        'RMSE': [],
        'MAE': [],
        'MAPE': [],
        'BIAS': []
    })
    for i in lista:
        # Lo leemos
        df = pd.read_csv(ruta+i, header=0, index_col=0, parse_dates=True)
        serie_nulo = df.squeeze('columns')
        
        # Imputamos con la media:
        serie_media = serie_nulo.fillna(serie_nulo.mean())

        # Calculamos la raiz del error cuadrático medio con las series imputadas con la media
        rmse_media = auxi.RMSE(serie, serie_nulo, serie_media)
        # Calculamos el error medio absoluto con las series imputadas con la media
        mae_media = auxi.MAE(serie, serie_nulo, serie_media)
        # Calculamos el error medio absoluto porcentual con las series imputadas con la media
        mape_media = auxi.MAPE(serie, serie_nulo, serie_media)
        # Calculamos el bias con las series imputadas con la media
        bias_media = auxi.BIAS(serie, serie_nulo, serie_media)

        nueva_fila = pd.DataFrame({ 'Dataset': [i], 'RMSE': [rmse_media], 'MAE': [mae_media], 'MAPE': [mape_media], 'BIAS': [bias_media]})
        dataframe_media = pd.concat([dataframe_media, nueva_fila], ignore_index=True)
    return dataframe_media

### Mediana

In [192]:
def imputar_mediana(serie, ruta, lista):
    dataframe_mediana = pd.DataFrame({
        'Dataset': [],
        'RMSE': [],
        'MAE': [],
        'MAPE': [],
        'BIAS': []
    })
    for i in lista:
        # Lo leemos
        df = pd.read_csv(ruta+i, header=0, index_col=0, parse_dates=True)
        serie_nulo = df.squeeze('columns')
        
        # Imputamos con la mediana
        serie_mediana = serie_nulo.fillna(serie_nulo.median())

        # Calculamos la raiz del error cuadrático medio con las series imputadas con la mediana
        rmse_mediana = auxi.RMSE(serie, serie_nulo, serie_mediana)
        # Calculamos el error medio absoluto con las series imputadas con la mediana
        mae_mediana = auxi.MAE(serie, serie_nulo, serie_mediana)
        # Calculamos el error medio absoluto porcentual con las series imputadas con la mediana
        mape_mediana = auxi.MAPE(serie, serie_nulo, serie_mediana)
        # Calculamos el bias con las series imputadas con la mediana
        bias_mediana = auxi.BIAS(serie, serie_nulo, serie_mediana)

        nueva_fila = pd.DataFrame({ 'Dataset': [i], 'RMSE': [rmse_mediana], 'MAE': [mae_mediana], 'MAPE': [mape_mediana], 'BIAS': [bias_mediana]})
        dataframe_mediana = pd.concat([dataframe_mediana, nueva_fila], ignore_index=True)
    return dataframe_mediana

### Interpolacion

In [193]:
def imputar_interpolacion(serie, ruta, lista):
    dataframe_inter = pd.DataFrame({
        'Dataset': [],
        'RMSE': [],
        'MAE': [],
        'MAPE': [],
        'BIAS': []
    })
    for i in lista:
        # Lo leemos
        df = pd.read_csv(ruta+i, header=0, index_col=0, parse_dates=True)
        serie_nulo = df.squeeze('columns')

        serie_inter = serie_nulo.copy()
        # Tratar problema de nulos al inicio, interpolate no los puede tratar bien
        parada=False
        j=0
        while(j<len(serie_inter) and parada!=True):
            valor = serie_inter.iloc[j]
            if np.isnan(valor):
                serie_inter.iloc[j] = serie_inter.mean()
            else:
                parada=True
            j+=1

        # Tratar problema de nulos al final, interpolate no los puede tratar bien
        parada=False
        j=len(serie_inter)-1
        while(j>=0 and parada!=True):
            valor = serie_inter.iloc[j]
            if np.isnan(valor):
                serie_inter.iloc[j] = serie_inter.mean()
            else:
                parada=True
            j-=1
        
        # Imputamos con interpolacion
        serie_inter = serie_inter.interpolate(method='linear')
        #print(serie_inter.head(3))

        # Calculamos la raiz del error cuadrático medio con las series imputadas con interpolacion
        rmse_inter = auxi.RMSE(serie, serie_nulo, serie_inter)
        # Calculamos el error medio absoluto con las series imputadas con interpolacion
        mae_inter = auxi.MAE(serie, serie_nulo, serie_inter)
        # Calculamos el error medio absoluto porcentual con las series imputadas con interpolacion
        mape_inter = auxi.MAPE(serie, serie_nulo, serie_inter)
        # Calculamos el bias con las series imputadas con interpolacion
        bias_inter = auxi.BIAS(serie, serie_nulo, serie_inter)

        nueva_fila = pd.DataFrame({ 'Dataset': [i], 'RMSE': [rmse_inter], 'MAE': [mae_inter], 'MAPE': [mape_inter], 'BIAS': [bias_inter]})
        dataframe_inter = pd.concat([dataframe_inter, nueva_fila], ignore_index=True)
    return dataframe_inter

## Listar

In [194]:
lista_datasets_temperatura = auxi.listar_datasets('..\miss_valued_data_temperatura')
lista_datasets_cardiovascular = auxi.listar_datasets('..\miss_valued_data_cardiovascular')
lista_datasets_sunspots = auxi.listar_datasets('..\miss_valued_data_manchas_solares')

## Imputamos media

In [195]:
dataframe_temperatura_media = imputar_media(series_temperatura, "..\miss_valued_data_temperatura\\", lista_datasets_temperatura)
dataframe_cardiovascular_media = imputar_media(series_cardiovascular, "..\miss_valued_data_cardiovascular\\", lista_datasets_cardiovascular)
dataframe_manchas_solares_media = imputar_media(series_sunspots, "..\miss_valued_data_manchas_solares\\", lista_datasets_sunspots)
dataframe_general_media = pd.concat([dataframe_temperatura_media, dataframe_cardiovascular_media, dataframe_manchas_solares_media], ignore_index=True)
dataframe_general_media

,Dataset,RMSE,MAE,MAPE,BIAS
0,min_temperature_bateria_10.csv,1.474758,0.377155,3.683127,-0.062002
1,min_temperature_bateria_15.csv,1.671954,0.519310,4.969444,-0.049206
2,min_temperature_bateria_20.csv,1.957098,0.716304,7.102209,-0.197963
3,min_temperature_error_medida_10.csv,1.306482,0.338414,3.160157,-0.000623
4,min_temperature_error_medida_15.csv,1.601408,0.505805,4.788038,-0.013763
5,min_temperature_error_medida_20.csv,1.790315,0.646521,6.087524,0.002636
6,min_temperature_interferencias_10.csv,1.350493,0.346774,3.197132,0.018786
7,min_temperature_interferencias_15.csv,1.637666,0.513872,4.811311,-0.021911
8,min_temperature_interferencias_20.csv,1.920747,0.696068,6.501651,0.006435
9,min_temperature_mix_10.csv,1.334668,0.337125,3.124871,-0.006962


## Imputamos mediana

In [196]:
dataframe_temperatura_mediana = imputar_mediana(series_temperatura, "..\miss_valued_data_temperatura\\", lista_datasets_temperatura)
dataframe_cardiovascular_mediana = imputar_mediana(series_cardiovascular, "..\miss_valued_data_cardiovascular\\", lista_datasets_cardiovascular)
dataframe_manchas_solares_mediana = imputar_mediana(series_sunspots, "..\miss_valued_data_manchas_solares\\", lista_datasets_sunspots)
dataframe_general_mediana = pd.concat([dataframe_temperatura_mediana, dataframe_cardiovascular_mediana, dataframe_manchas_solares_mediana], ignore_index=True)
dataframe_general_mediana

,Dataset,RMSE,MAE,MAPE,BIAS
0,min_temperature_bateria_10.csv,1.469475,0.374877,3.667503,-0.047850
1,min_temperature_bateria_15.csv,1.668904,0.517347,4.956604,-0.029915
2,min_temperature_bateria_20.csv,1.949547,0.712610,7.074714,-0.182462
3,min_temperature_error_medida_10.csv,1.306689,0.337733,3.155294,0.007393
4,min_temperature_error_medida_15.csv,1.601013,0.504655,4.780325,0.000219
5,min_temperature_error_medida_20.csv,1.792323,0.645728,6.086141,0.038007
6,min_temperature_interferencias_10.csv,1.353682,0.347015,3.202149,0.034858
7,min_temperature_interferencias_15.csv,1.636779,0.513116,4.807837,-0.006709
8,min_temperature_interferencias_20.csv,1.922886,0.695263,6.501589,0.041046
9,min_temperature_mix_10.csv,1.335002,0.336323,3.121034,0.011733


## Imputamos interpolación

In [197]:
dataframe_temperatura_inter = imputar_interpolacion(series_temperatura, "..\miss_valued_data_temperatura\\", lista_datasets_temperatura)
dataframe_cardiovascular_inter = imputar_interpolacion(series_cardiovascular, "..\miss_valued_data_cardiovascular\\", lista_datasets_cardiovascular)
dataframe_manchas_solares_inter = imputar_interpolacion(series_sunspots, "..\miss_valued_data_manchas_solares\\", lista_datasets_sunspots)
dataframe_general_inter = pd.concat([dataframe_temperatura_inter, dataframe_cardiovascular_inter, dataframe_manchas_solares_inter], ignore_index=True)
dataframe_general_inter

,Dataset,RMSE,MAE,MAPE,BIAS
0,min_temperature_bateria_10.csv,1.032988,0.256995,2.817399,-0.004436
1,min_temperature_bateria_15.csv,1.369825,0.425133,4.260352,-0.135515
2,min_temperature_bateria_20.csv,1.656718,0.591620,6.090818,-0.198220
3,min_temperature_error_medida_10.csv,0.693817,0.170934,1.797728,0.011980
4,min_temperature_error_medida_15.csv,0.845862,0.251451,2.630531,0.014389
5,min_temperature_error_medida_20.csv,0.999040,0.341294,3.496302,0.020872
6,min_temperature_interferencias_10.csv,0.746303,0.182032,1.791032,-0.012185
7,min_temperature_interferencias_15.csv,0.929418,0.280230,2.767899,-0.023768
8,min_temperature_interferencias_20.csv,1.134056,0.389091,3.894535,0.023918
9,min_temperature_mix_10.csv,0.842076,0.198999,1.931381,-0.027683
